# Лабораторная работа 1. Сигналы и звуки



In [2]:
import os

In [3]:
from thinkdsp import read_wave

wave = read_wave('e:/wav/Sound_01822.wav')
wave.normalize()
wave.make_audio()

ModuleNotFoundError: No module named 'thinkdsp'

In [ ]:
wave.plot()

In [ ]:
segment = wave.segment(start=1.1, duration=0.3)
segment.make_audio()

In [ ]:
segment.plot()

In [ ]:
segment.segment(start=1.1, duration=0.005).plot()

In [ ]:
spectrum = segment.make_spectrum()
spectrum.plot(high=7000)

In [ ]:
spectrum = segment.make_spectrum()
spectrum.plot(high=1000)

In [ ]:
spectrum.peaks()[:30]

In [ ]:
spectrum.low_pass(2000)

In [ ]:
spectrum.make_wave().make_audio()

In [ ]:
from thinkdsp import decorate
from IPython.display import display

def filter_wave(wave, start, duration, cutoff):
    """Selects a segment from the wave and filters it.

    Plots the spectrum and displays an Audio widget.

    wave: Wave object
    start: time in s
    duration: time in s
    cutoff: frequency in Hz
    """
    segment = wave.segment(start, duration)
    spectrum = segment.make_spectrum()

    spectrum.plot(high=5000, color='0.7')
    spectrum.low_pass(cutoff)
    spectrum.plot(high=5000, color='#045a8d')
    decorate(xlabel='Frequency (Hz)')

    audio = spectrum.make_wave().make_audio()
    display(audio)

In [ ]:
from ipywidgets import interact, fixed

interact(filter_wave, wave=fixed(wave),
         start=(0, 5, 0.1), duration=(0, 5, 0.1), cutoff=(0, 5000, 100));

In [ ]:
from thinkdsp import SinSignal

signal = (SinSignal(freq=200, amp=1.0) +
          SinSignal(freq=400, amp=0.5) +
          SinSignal(freq=600, amp=0.25))
signal.plot()

In [ ]:
wave2 = signal.make_wave(duration=1)
wave2.apodize()

In [ ]:
wave2.make_audio()

In [ ]:
spectrum = wave2.make_spectrum()
spectrum.plot(high=1000)

In [ ]:
signal += SinSignal(freq=450)
signal.make_wave().make_audio()

3 Упражнение

In [ ]:
wave3 = read_wave('e:/wav/Sound_01822.wav')
wave3.normalize()
wave3.make_audio()

In [ ]:
def stretch(wave, factor):
    wave.ts *= factor
    wave.framerate /= factor

In [ ]:
stretch(wave3, 0.5)
wave3.make_audio()

In [ ]:
wave3.plot()

# Лабораторная работа 2. Гармоники


In [ ]:
import os
from thinkdsp import decorate

In [ ]:
from thinkdsp import Sinusoid
from thinkdsp import normalize, unbias
import numpy as np

class SawtoothSignal(Sinusoid):
  def evaluate(self, ts):
        cycles = self.freq * ts + self.offset / np.pi / 2
        frac, _ = np.modf(cycles)
        ys = normalize(unbias(frac), self.amp)
        return ys

In [ ]:
sawtooth = SawtoothSignal().make_wave(duration=0.6, framerate=36500)
sawtooth.make_audio()

In [ ]:
sawtooth.make_spectrum().plot()
decorate(xlabel='Frequency (Hz)')

In [ ]:
from thinkdsp import SquareSignal

sawtooth.make_spectrum().plot(color='red')
square = SquareSignal(amp=0.5).make_wave(duration=0.6, framerate=36500)
square.make_spectrum().plot()
decorate(xlabel='Frequency (Hz)')

In [ ]:
from thinkdsp import TriangleSignal

sawtooth.make_spectrum().plot(color='green')
triangle = TriangleSignal(amp=0.8).make_wave(duration=0.6, framerate=36500)
triangle.make_spectrum().plot()
decorate(xlabel='Frequency (Hz)')

Гармоники треугольной волны затухают пропорционально 1/f^2, а пилообразные затухают как 1/f

Упражнение 2

In [ ]:
square = SquareSignal(1500).make_wave(duration=0.7, framerate=15000)

In [ ]:
square.make_spectrum().plot()
decorate(xlabel='Frequency (Hz)')

Создали треугольный сигнал и посмотрели на его спектр

In [ ]:
square.make_audio()

Сравним с 1000hz

In [ ]:
from thinkdsp import SinSignal

SinSignal(1000).make_wave(duration=0.7, framerate=15000).make_audio()

Упражнение 3

In [ ]:
triangle = TriangleSignal().make_wave(duration=0.005)
triangle.plot()
decorate(xlabel='Time (sec)')

In [ ]:
spectrum = triangle.make_spectrum()
spectrum.hs[0]

In [ ]:
spectrum.hs[0] = 150
triangle.plot(color='red')
spectrum.make_wave().plot()
decorate(xlabel='Time (s)')

Компонент нулевой частоты представляет собой сумму всех значений в сигнале, если сигнал несмещен, компонент нулевой частоты равен 0

Упражнение 4

In [ ]:
def filter_spectrum(spectrum):
    spectrum.hs[1:] /= spectrum.fs[1:]
    spectrum.hs[0] = 0

In [ ]:
wave = TriangleSignal(freq=450).make_wave(duration=0.7)
wave.make_audio()

Масштабируем изображение

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot(high=15000, color='red')
filter_spectrum(spectrum)
spectrum.scale(450)
spectrum.plot(high=15000)
decorate(xlabel='Frequency (Hz)')

Фильтр подавляет гармоники, поэтому он действует как фильтр нижних частот.

In [ ]:
filtered = spectrum.make_wave()
filtered.make_audio()

Упраженение 5

In [ ]:
freq = 600
signal = SawtoothSignal(freq=freq)
wave = signal.make_wave(duration=0.7, framerate=25000)
wave.make_audio()

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot()
decorate(xlabel='Frequency (Hz)')


Если мы применим фильтр, который мы написали в предыдущем упражнении, мы можем уменьшить гармоники

In [ ]:
spectrum.plot(color='red')
filter_spectrum(spectrum)
spectrum.scale(freq)
spectrum.plot()
decorate(xlabel='Frequency (Hz)')

In [ ]:
wave = spectrum.make_wave()
wave.make_audio()

In [ ]:
wave.segment(duration=0.02).plot()
decorate(xlabel='Time (sec)')

Другой подход заключается в суммировании серии косинусоидальных сигналов с правильными частотами и амплитудами.

In [ ]:
from thinkdsp import CosSignal

freqs = np.arange(500, 9500, 500)
amps = 1 / freqs**2

signal = sum(CosSignal(freq, amp) for freq, amp in zip(freqs, amps))
signal

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot()
decorate(xlabel='Frequency (Hz)')

In [ ]:
wave = signal.make_wave(duration=0.7, framerate=25000)
wave.make_audio()

In [ ]:
wave.segment(duration=0.02).plot()
decorate(xlabel='Time (sec)')

Есть некая схожесть с пароболическими сигналами

In [ ]:
from thinkdsp import ParabolicSignal

wave = ParabolicSignal(freq=600).make_wave(duration=0.7, framerate=25000)
wave.make_audio()

In [ ]:
wave.segment(duration=0.02).plot()
decorate(xlabel='Time (sec)')

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot()
decorate(xlabel='Frequency (Hz)')

# Лабораторная работа 3. Апериодические сигналы

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from thinkdsp import decorate

In [ ]:
from thinkdsp import SinSignal

signal = SinSignal(freq=550)
duration = signal.period * 30.5
wave = signal.make_wave(duration)
spectrum = wave.make_spectrum()

In [ ]:
spectrum.plot(high=1000)
decorate(xlabel='Frequency (Hz)')

In [ ]:
for window_func in [np.bartlett, np.blackman, np.hamming, np.hanning]:
    wave = signal.make_wave(duration)
    wave.ys *= window_func(len(wave.ys))

    spectrum = wave.make_spectrum()
    spectrum.plot(high=1000, label=window_func.__name__)

decorate(xlabel='Frequency (Hz)')

Все четыре хорошо справляются с уменьшением утечек. Фильтр Бартлетта оставляет некоторый остаточный «звон». Фильтр Хэмминга рассеивает наименьшее количество энергии.

Упражнение 2

In [ ]:
from thinkdsp import Chirp
from thinkdsp import normalize, unbias

PI2 = 2 * np.pi

class SawtoothChirp(Chirp):
   def evaluate(self, ts):
        freqs = np.linspace(self.start, self.end, len(ts))
        dts = np.diff(ts, prepend=0)
        dphis = PI2 * freqs * dts
        phases = np.cumsum(dphis)
        cycles = phases / PI2
        frac, _ = np.modf(cycles)
        ys =  normalize(unbias(frac), self.amp)
        return ys

In [ ]:
signal = SawtoothChirp(start=220, end=880)
wave = signal.make_wave(duration=1, framerate=6000)
wave.apodize()
wave.make_audio()

In [ ]:
sp = wave.make_spectrogram(512)
sp.plot(high=1000)
decorate(xlabel='Time (s)', ylabel='Frequency (Hz)')

При относительно низкой частоте кадров можно увидеть искаженные гармоники, отражающиеся от частоты сворачивания. И можно услышать их как фоновое шипение. Если увеличить частоту кадров - они исчезают.

Пилообразный писк можно распознать как сигнал тревоги.

In [ ]:
from thinkdsp import read_wave

read_wave('E:\wav\code_tos-redalert.wav').make_audio()

Упражнение 3

In [ ]:
signal = SawtoothChirp(start=2500, end=3000)
wave = signal.make_wave(duration=1, framerate=20000)
wave.make_audio()

In [ ]:
wave.make_spectrum().plot()
decorate(xlabel='Frequency (Hz)')

Упражнение 4

In [ ]:
wave = read_wave('E:\wav\code_72475__rockwehrmann__glissup02.wav')
wave.make_audio()

In [ ]:
wave.make_spectrogram(512).plot(high=5000)
decorate(xlabel='Time (sec)', ylabel='Frequency (Hz)')

Упражнение 5

In [ ]:
class TromboneGliss(Chirp):
  def evaluate(self, ts):
        l1, l2 = 1.0 / self.start, 1.0 / self.end
        lengths = np.linspace(l1, l2, len(ts))
        freqs = 1 / lengths

        dts = np.diff(ts, prepend=0)
        dphis = PI2 * freqs * dts
        phases = np.cumsum(dphis)
        ys = self.amp * np.cos(phases)
        return ys

1 часть

In [ ]:
low = 256
high = 356
signal = TromboneGliss(high, low)
wave1 = signal.make_wave(duration=1)
wave1.apodize()
wave1.make_audio()

2 часть

In [ ]:
signal = TromboneGliss(low, high)
wave2 = signal.make_wave(duration=1)
wave2.apodize()
wave2.make_audio()

Соединим их вместе

In [ ]:
wave = wave1 | wave2
wave.make_audio()

In [ ]:
sp = wave.make_spectrogram(1024)
sp.plot(high=1000)
decorate(xlabel='Time (sec)', ylabel='Frequency (Hz)')

Упражнение 6

In [ ]:
wave = read_wave('E:\wav\code_voice.wav')
wave.make_audio()

In [ ]:
wave.make_spectrogram(1024).plot(high=1000)
decorate(xlabel='Time (sec)', ylabel='Frequency (Hz)')

Полоса внизу, вероятно, является фоновым шумом. Пики на спектрограмме называются формантами.

In [ ]:
high = 1000

segment = wave.segment(start=1, duration=0.25)
segment.make_spectrum().plot(high=high)

Основная частота около 100 Гц. Следующие самые высокие пики находятся на частотах 200 Гц и 700 Гц

In [ ]:
segment = wave.segment(start=2.2, duration=0.25)
segment.make_spectrum().plot(high=high)
decorate(xlabel='Frequency (Hz)')

In [ ]:
segment = wave.segment(start=3.5, duration=0.25)
segment.make_spectrum().plot(high=high)
decorate(xlabel='Frequency (Hz)')

In [ ]:
segment = wave.segment(start=5.1, duration=0.25)
segment.make_spectrum().plot(high=high)
decorate(xlabel='Frequency (Hz)')

In [ ]:
segment = wave.segment(start=6.5, duration=0.25)
segment.make_spectrum().plot(high=high)
decorate(xlabel='Frequency (Hz)')

# Лабораторная работа 4. Шум


In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

from thinkdsp import decorate

In [ ]:
from thinkdsp import read_wave

wave = read_wave('E:\wav\code_spoonbender_wind-through-trees-3b.wav')
wave.make_audio()

In [ ]:
segment = wave.segment(start=1.5, duration=1.0)
segment.make_audio()

In [ ]:
spectrum = segment.make_spectrum()
spectrum.plot_power()
decorate(xlabel='Frequency (Hz)',
         ylabel='Power')

In [ ]:
spectrum.plot_power()

loglog = dict(xscale='log', yscale='log')
decorate(xlabel='Frequency (Hz)',
         ylabel='Power',
         **loglog)

In [ ]:
segment2 = wave.segment(start=2.5, duration=1.0)
segment2.make_audio()

In [ ]:
spectrum2 = segment2.make_spectrum()

spectrum.plot_power(alpha=0.5)
spectrum2.plot_power(alpha=0.5)
decorate(xlabel='Frequency (Hz)',
         ylabel='Power')

In [ ]:
spectrum.plot_power(alpha=0.5)
spectrum2.plot_power(alpha=0.5)
decorate(xlabel='Frequency (Hz)',
         ylabel='Power',
         **loglog)

In [ ]:
segment.make_spectrogram(512).plot(high=5000)
decorate(xlabel='Time(s)', ylabel='Frequency (Hz)')

Реализуем метод Бартлетта и используем его, чтобы оценить спектр мощности для шумовой волны

In [ ]:
from thinkdsp import Spectrum

def bartlett_method(wave, seg_length=512, win_flag=True):

    spectro = wave.make_spectrogram(seg_length, win_flag)
    spectrums = spectro.spec_map.values()

    psds = [spectrum.power for spectrum in spectrums]

    hs = np.sqrt(sum(psds) / len(psds))
    fs = next(iter(spectrums)).fs

    spectrum = Spectrum(hs, fs, wave.framerate)
    return spectrum

In [ ]:
psd = bartlett_method(segment)
psd2 = bartlett_method(segment2)

psd.plot_power()
psd2.plot_power()

decorate(xlabel='Frequency (Hz)',
         ylabel='Power',
         **loglog)

Упражнение 2

In [ ]:
import pandas as pd

df = pd.read_csv('E:\wav\BTC_USD_2013-10-01_2023-04-14-CoinDesk.csv',
                 parse_dates=[0])
df

In [ ]:
ys = df['Closing Price (USD)']
ts = df.index

In [ ]:
from thinkdsp import Wave

wave = Wave(ys, ts, framerate=1)
wave.plot()
decorate(xlabel='Time (days)')

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot_power()
decorate(xlabel='Frequency (1/days)',
         ylabel='Power',
         **loglog)

In [ ]:
spectrum.estimate_slope()[0]

Красный шум должен иметь наклон -2. Наклон этой PSD близок к 1,7, поэтому трудно сказать, следует ли считать это красным шумом или же это разновидность розового шума


Упражнение 3

In [ ]:
from thinkdsp import Noise

class UncorrelatedPoissonNoise(Noise):

    def evaluate(self, ts):

        ys = np.random.poisson(self.amp, len(ts))
        return ys

In [ ]:
amp = 0.001
framerate = 10000
duration = 1

signal = UncorrelatedPoissonNoise(amp=amp)
wave = signal.make_wave(duration=duration, framerate=framerate)
wave.make_audio()

In [ ]:
expected = amp * framerate * duration
actual = sum(wave.ys)
print(expected, actual)

In [ ]:
wave.plot()

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot_power()
decorate(xlabel='Frequency (Hz)',
         ylabel='Power',
         **loglog)

In [ ]:
spectrum.estimate_slope().slope

Похоже на белый шум, а наклон близок к 0.

In [ ]:
amp = 1
framerate = 10000
duration = 1

signal = UncorrelatedPoissonNoise(amp=amp)
wave = signal.make_wave(duration=duration, framerate=framerate)
wave.make_audio()

In [ ]:
wave.plot()

In [ ]:
import matplotlib.pyplot as plt

def normal_prob_plot(sample, fit_color='0.8', **options):

    n = len(sample)
    xs = np.random.normal(0, 1, n)
    xs.sort()

    ys = np.sort(sample)

    mean, std = np.mean(sample), np.std(sample)
    fit_ys = mean + std * xs
    plt.plot(xs, fit_ys, color='gray', alpha=0.5, label='model')

    plt.plot(xs, ys, **options)

In [ ]:
spectrum = wave.make_spectrum()
spectrum.hs[0] = 0

normal_prob_plot(spectrum.real, label='real')
decorate(xlabel='Normal sample',
        ylabel='Power')

In [ ]:
normal_prob_plot(spectrum.imag, label='imag', color='C2')
decorate(xlabel='Normal sample')

4 упражнение

Основная идея этого алгоритма состоит в том, чтобы сложить несколько последовательностей случайных чисел, которые обновляются с разной частотой дискретизации. Первый источник должен обновляться на каждом временном шаге; второй источник через каждый второй временной шаг, третий источник через каждый четвертый шаг и так далее.

In [ ]:
nrows = 100
ncols = 5

array = np.empty((nrows, ncols))
array.fill(np.nan)
array[0, :] = np.random.random(ncols)
array[:, 0] = np.random.random(nrows)
array[0:6]

Следующим шагом является выбор мест, где случайные источники меняются.

In [ ]:
p = 0.5
n = nrows
cols = np.random.geometric(p, n)
cols[cols >= ncols] = 0
cols

In [ ]:
rows = np.random.randint(nrows, size=n)
rows

In [ ]:
array[rows, cols] = np.random.random(n)
array[0:6]

In [ ]:
df = pd.DataFrame(array)
df.head()

In [ ]:
filled = df.fillna(method='ffill', axis=0)
filled.head()

In [ ]:
total = filled.sum(axis=1)
total.head()

In [ ]:
wave = Wave(total.values)
wave.plot()

Вот весь процесс в функции:


In [ ]:
def voss(nrows, ncols=16):

    array = np.empty((nrows, ncols))
    array.fill(np.nan)
    array[0, :] = np.random.random(ncols)
    array[:, 0] = np.random.random(nrows)

    n = nrows
    cols = np.random.geometric(0.5, n)
    cols[cols >= ncols] = 0
    rows = np.random.randint(nrows, size=n)
    array[rows, cols] = np.random.random(n)

    df = pd.DataFrame(array)
    df.fillna(method='ffill', axis=0, inplace=True)
    total = df.sum(axis=1)

    return total.values

In [ ]:
ys = voss(11040)
ys

In [ ]:
wave = Wave(ys)
wave.unbias()
wave.normalize()

In [ ]:
wave.plot()

Как и ожидалось, он больше похож на случайное блуждание, чем белый шум, но выглядит более случайным, чем красный шум.

In [ ]:
wave.make_audio()

In [ ]:
spectrum = wave.make_spectrum()
spectrum.hs[0] = 0
spectrum.plot_power()
decorate(xlabel='Frequency (Hz)',
         ylabel='Power',
         **loglog)

In [ ]:
spectrum.estimate_slope().slope

In [ ]:
seg_length = 64 * 1024
iters = 100
wave = Wave(voss(seg_length * iters))
len(wave)

In [ ]:
spectrum = bartlett_method(wave, seg_length=seg_length, win_flag=False)
spectrum.hs[0] = 0
len(spectrum)

In [ ]:
spectrum.plot_power()
decorate(xlabel='Frequency (Hz)',
         ylabel='Power',
         **loglog)

In [ ]:
spectrum.estimate_slope().slope

наклон близок к -1.

# Лабораторная работа 5. Автокорреляция

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

from thinkdsp import decorate

In [ ]:
import pandas as pd

df = pd.read_csv('E:\wav\BTC_USD_2013-10-01_2023-04-14-CoinDesk.csv',
                 parse_dates=[0])

ys = df['Closing Price (USD)']
ts = df.index

In [ ]:
from thinkdsp import Wave

wave = Wave(ys, ts, framerate=1)
wave.plot()
decorate(xlabel='Time (days)',
         ylabel='Price of BitCoin ($)')

Вот функция автокорреляции, использующая статистическое определение, которое устраняет смещения, нормализует и стандартизирует; то есть он сдвигает среднее значение к нулю, делит на стандартное отклонение и делит сумму на N.

In [ ]:
def autocorr(wave):

    lags = np.arange(len(wave.ys)//2)
    corrs = [serial_corr(wave, lag) for lag in lags]
    return lags, corrs

In [ ]:
def serial_corr(wave, lag=1):

    n = len(wave)
    y1 = wave.ys[lag:]
    y2 = wave.ys[:n-lag]
    corr_mat = np.corrcoef(y1, y2)
    return corr_mat[0, 1]

In [ ]:
lags, corrs = autocorr(wave)
plt.plot(lags, corrs)
decorate(xlabel='Lag',
         ylabel='Correlation')

In [ ]:
N = len(wave)
corrs2 = np.correlate(wave.ys, wave.ys, mode='same')
lags = np.arange(-N//2, N//2)
plt.plot(lags, corrs2)
decorate(xlabel='Lag',
         ylabel='Dot product')

In [ ]:
N = len(corrs2)
half = corrs2[N//2:]
plt.plot(half)
decorate(xlabel='Lag',
         ylabel='Dot product')

In [ ]:
lengths = range(N, N//2, -1)
half /= lengths
half /= half[0]
plt.plot(half)
decorate(xlabel='Lag',
         ylabel='Dot product')

In [ ]:
plt.plot(corrs, label='autocorr')
plt.plot(half, label='correlate')
decorate(xlabel='Lag', ylabel='Correlation')

Даже после стандартизации результаты выглядят существенно иначе.

Для этого набора данных, вероятно, более подходящим является статистическое определение АКФ.

Упражнение 2

In [ ]:
from thinkdsp import read_wave

wave = read_wave('E:\wav\code_28_bcjordan_voicedownbew.wav')
wave.normalize()
wave.make_audio()

In [ ]:
wave.make_spectrogram(2048).plot(high=4200)
decorate(xlabel='Time (s)',
         ylabel='Frequency (Hz)')

In [ ]:
def estimate_fundamental(segment, low=70, high=150):
    lags, corrs = autocorr(segment)
    lag = np.array(corrs[low:high]).argmax() + low
    period = lag / segment.framerate
    frequency = 1 / period
    return frequency

In [ ]:
duration = 0.01
segment = wave.segment(start=0.2, duration=duration)
freq = estimate_fundamental(segment)
freq

In [ ]:
step = 0.05
starts = np.arange(0.0, 1.4, step)

ts = []
freqs = []

for start in starts:
    ts.append(start + step/2)
    segment = wave.segment(start=start, duration=duration)
    freq = estimate_fundamental(segment)
    freqs.append(freq)

In [ ]:
wave.make_spectrogram(2048).plot(high=900)
plt.plot(ts, freqs, color='white')
decorate(xlabel='Time (s)',
                     ylabel='Frequency (Hz)')

# Лабораторная работа 6. Дискретное косинусовое преобразование

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os

In [ ]:
from thinkdsp import decorate

In [ ]:
from thinkdsp import UncorrelatedGaussianNoise

signal = UncorrelatedGaussianNoise()
noise = signal.make_wave(duration=1.0, framerate=8192)
noise.ys.shape

In [ ]:
from scipy.stats import linregress

loglog = dict(xscale='log', yscale='log')

def plot_bests(ns, bests):
    plt.plot(ns, bests)
    decorate(**loglog)

    x = np.log(ns)
    y = np.log(bests)
    t = linregress(x,y)
    slope = t[0]

    return slope

In [ ]:
PI2 = np.pi * 2

def analyze1(ys, fs, ts):

    args = np.outer(ts, fs)
    M = np.cos(PI2 * args)
    amps = np.linalg.solve(M, ys)
    return amps

In [ ]:
ns = 2 ** np.arange(6, 13)
ns

In [ ]:
results = []
for N in ns:
  print(N)
  ts = (0.5 + np.arange(N)) / N
  freqs = (0.5 + np.arange(N)) / 2
  ys = noise.ys[:N]
  result = %timeit -r1 -o analyze1(ys, freqs, ts)
  results.append(result)

bests = [result.best for result in results]
plot_bests(ns, bests)

Как говорится в пособии chap06starter расчетный уклон ближе к степени двойки, чем тройки. Производительность np.linalg.solve почти квадратична в этом диапозоне размеров массива. К сожеланию, из-за малой мощности рабочей машины, не удалось протестировать analyze1 со заничением 16384. Из графика видно, что линия в конце чуть-чуть изогнута, что говорит о том, что размер массива не достигнут, где analyze1 показывает рост пропорционально  n3 . Из этого можно сделать вывод, что при больших размерах массива, рост analyze1 будет пропорционален  n3 .

In [ ]:
def analyze2(ys, fs, ts):

    args = np.outer(ts, fs)
    M = np.cos(PI2 * args)
    amps = np.dot(M, ys) / 2
    return amps

In [ ]:
signal = UncorrelatedGaussianNoise()
noise = signal.make_wave(duration=1.0, framerate=16384)
noise.ys.shape

ns = 2 ** np.arange(6, 15)
ns

In [ ]:
results = []
for N in ns:
  print(N)
  ts = (0.5 + np.arange(N)) / N
  freqs = (0.5 + np.arange(N)) / 2
  ys = noise.ys[:N]
  result = %timeit -r1 -o analyze2(ys, freqs, ts)
  results.append(result)

bests2 = [result.best for result in results]
plot_bests(ns, bests2)

Как видно из графика, analyze2 растет пропорционально  n2 . Проведем такой же эксперимент scipy.fftpack.dct.

In [ ]:
import scipy.fftpack

def scipy_dct(ys, freqs, ts):
    return scipy.fftpack.dct(ys, type=3)

In [ ]:
results = []
for N in ns:
  ys = noise.ys[:N]
  result = %timeit -r1 -o scipy.fftpack.dct(ys, type=3)
  results.append(result)

bests3 = [result.best for result in results]
plot_bests(ns, bests3)

dct реализация намного быстрее, она пропорциональна nlogn

Упражнение 2

In [ ]:
from thinkdsp import read_wave

wave = read_wave('E:\wav\code_saxop.wav')
wave.make_audio()

In [ ]:
segment = wave.segment(start=1.2, duration=0.5)
segment.normalize()
segment.make_audio()

In [ ]:
seg_dct = segment.make_dct()
seg_dct.plot(high=4000)
decorate(xlabel='Частота (Гц)', ylabel='DCT')

Из графика видно, что есть немного частот с большой амплитудой. Следующая функция compress и режет элемент, которые ниже аргумента thresh.

In [ ]:
def compress(dct, thresh=1):
    count = 0
    for i, amp in enumerate(dct.amps):
        if np.abs(amp) < thresh:
            dct.hs[i] = 0
            count += 1

    n = len(dct.amps)
    print(count, n, 100 * count / n, sep='\t')

In [ ]:
seg_dct = segment.make_dct()
compress(seg_dct, thresh=10)
seg_dct.plot(high=4000)

In [ ]:
seg2 = seg_dct.make_wave()
seg2.make_audio()

# Лабораторная работа 7. Дискретное преобразование Фурье

In [ ]:
import numpy as np
PI2 = 2 * np.pi

In [ ]:
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)
print(hs)

In [ ]:
def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

In [ ]:
hs2 = dft(ys)
np.sum(np.abs(hs - hs2))

In [ ]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])

    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)

    return np.tile(He, 2) + W * np.tile(Ho, 2)

In [ ]:
hs3 = fft_norec(ys)
np.sum(np.abs(hs - hs3))

In [ ]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys

    He = fft(ys[::2])
    Ho = fft(ys[1::2])

    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)

    return np.tile(He, 2) + W * np.tile(Ho, 2)

In [ ]:
hs4 = fft(ys)
np.sum(np.abs(hs - hs4))

Эта реализация БПФ требует времени, пропорционального nlogn. Он также занимает место, пропорциональное nlogn , и тратит некоторое время на создание и копирование массивов. Его можно улучшить для запуска «на месте». В этом случае он не требует дополнительного места и тратит меньше времени на накладные расходы.

# Лабораторная работа 8. Фильтрация и свертка

In [ ]:
import os

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from thinkdsp import decorate

In [ ]:
import scipy.signal

gaussian = scipy.signal.gaussian(M=32, std=2)
gaussian /= sum(gaussian)
plt.plot(gaussian)
decorate(xlabel='Index')

In [ ]:
fft_gaussian = np.fft.fft(gaussian)
plt.plot(abs(fft_gaussian))
decorate(xlabel='Frequency (Hz)', ylabel='Amplitude')

Если мы прокрутим отрицательные частоты влево, мы сможем более четко увидеть, что это гауссово движение, по крайней мере, приблизительно

In [ ]:
N = len(gaussian)
fft_rolled = np.roll(fft_gaussian, N//2)
plt.plot(abs(fft_rolled))
decorate(xlabel='Frequency (Hz)', ylabel='Amplitude')

In [ ]:
def plot_gaussian(std):
    M = 32
    gaussian = scipy.signal.gaussian(M=M, std=std)
    gaussian /= sum(gaussian)

    plt.subplot(1, 2, 1)
    plt.plot(gaussian)
    decorate(xlabel='Time')

    fft_gaussian = np.fft.fft(gaussian)
    fft_rolled = np.roll(fft_gaussian, M//2)

    plt.subplot(1, 2, 2)
    plt.plot(np.abs(fft_rolled))
    decorate(xlabel='Frequency')
    plt.show()

plot_gaussian(2)

In [ ]:
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

slider = widgets.FloatSlider(min=0.1, max=10, value=2)
interact(plot_gaussian, std=slider);

Упражнение 2

In [ ]:
from thinkdsp import SquareSignal

signal = SquareSignal(freq=440)
wave = signal.make_wave(duration=1.0, framerate=44100)

In [ ]:
M = 15
std = 2.5

gaussian = scipy.signal.gaussian(M=M, std=std)
bartlett = np.bartlett(M)
blackman = np.blackman(M)
hamming = np.hamming(M)
hanning = np.hanning(M)

windows = [blackman, gaussian, hanning, hamming]
names = ['blackman', 'gaussian', 'hanning', 'hamming']

for window in windows:
    window /= sum(window)

In [ ]:
for window, name in zip(windows, names):
    plt.plot(window, label=name)

decorate(xlabel='Index')

In [ ]:
def zero_pad(array, n):
    """Extends an array with zeros.

    array: NumPy array
    n: length of result

    returns: new NumPy array
    """
    res = np.zeros(n)
    res[:len(array)] = array
    return res

In [ ]:
def plot_window_dfts(windows, names):
    """
    """
    for window, name in zip(windows, names):
        padded =  zero_pad(window, len(wave))
        dft_window = np.fft.rfft(padded)
        plt.plot(abs(dft_window), label=name)

 Хэмминг падает быстрее всего, Блэкман — медленнее, а Хэннинг имеет самые заметные боковые лепестки.

In [ ]:
plot_window_dfts(windows, names)
decorate(xlabel='Frequency (Hz)')

In [ ]:
plot_window_dfts(windows, names)
decorate(xlabel='Frequency (Hz)', yscale='log')


В логарифмическом масштабе мы видим, что показатели Хэмминга и Ханнинга поначалу снижаются быстрее, чем у двух других.

# Лабораторная работа 9. Дифференцирование и интегрирование

In [ ]:
import os

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from thinkdsp import decorate

Треугольный сигнал

In [ ]:
from thinkdsp import TriangleSignal

in_wave = TriangleSignal(freq=50).make_wave(duration=0.1, framerate=44200)
in_wave.plot()
decorate(xlabel='Time (sec)')

In [ ]:
out_wave = in_wave.diff()
out_wave.plot()
decorate(xlabel='Time (sec)')

In [ ]:
out_wave2 = in_wave.make_spectrum().differentiate().make_wave()
out_wave2.plot()
decorate(xlabel='Time (sec)')

Упражнение 2

Квадратный сигнал

In [ ]:
from thinkdsp import SquareSignal

in_wave = SquareSignal(freq=50).make_wave(duration=0.1, framerate=44100)
in_wave.plot()
decorate(xlabel='Time (sec)')

In [ ]:
out_wave = in_wave.cumsum()
out_wave.plot()
decorate(xlabel='Time (sec)')

In [ ]:
spectrum = in_wave.make_spectrum().integrate()
spectrum.hs[0] = 0
out_wave2 = spectrum.make_wave()
out_wave2.plot()
decorate(xlabel='Time (sec)')

In [ ]:
out_wave.unbias()
out_wave.normalize()
out_wave2.normalize()
out_wave.plot()
out_wave2.plot()

In [ ]:
out_wave.max_diff(out_wave2)

Упражнение 3

In [ ]:
from thinkdsp import SawtoothSignal

in_wave = SawtoothSignal(freq=50).make_wave(duration=0.1, framerate=44100)
in_wave.plot()
decorate(xlabel='Time (s)')

Первая кумулятивная сумма зубца пилы – это парабола:

In [ ]:
out_wave = in_wave.cumsum()
out_wave.unbias()
out_wave.plot()
decorate(xlabel='Time (sec)')

In [ ]:
out_wave = out_wave.cumsum()
out_wave.plot()
decorate(xlabel='Time (sec)')

In [ ]:
spectrum = in_wave.make_spectrum().integrate().integrate()
spectrum.hs[0] = 0
out_wave2 = spectrum.make_wave()
out_wave2.plot()
decorate(xlabel='Time (sec)')

In [ ]:
out_wave2.make_spectrum().plot(high=500)

В этот момент результат все больше и больше напоминает синусоиду.

Упражнение 4

Кубический сигнал

In [ ]:
from thinkdsp import CubicSignal

in_wave = CubicSignal(freq=0.0005).make_wave(duration=10000, framerate=1)
in_wave.plot()

In [ ]:
out_wave = in_wave.diff()
out_wave.plot()

In [ ]:
out_wave = out_wave.diff()
out_wave.plot()

In [ ]:
spectrum = in_wave.make_spectrum().differentiate().differentiate()
out_wave2 = spectrum.make_wave()
out_wave2.plot()
decorate(xlabel='Time (sec)')

In [ ]:
from thinkdsp import zero_pad
from thinkdsp import Wave

diff_window = np.array([-1.0, 2.0, -1.0])
padded = zero_pad(diff_window, len(in_wave))
diff_wave = Wave(padded, framerate=in_wave.framerate)
diff_filter = diff_wave.make_spectrum()
diff_filter.plot(label='2nd diff')

decorate(xlabel='Frequency (Hz)',
                 ylabel='Amplitude ratio')

In [ ]:
PI2 = np.pi * 2

deriv_filter = in_wave.make_spectrum()
deriv_filter.hs = (PI2 * 1j * deriv_filter.fs)**2
deriv_filter.plot(label='2nd deriv')

decorate(xlabel='Frequency (Hz)',
                 ylabel='Amplitude ratio')

In [ ]:
diff_filter.plot(label='2nd diff')
deriv_filter.plot(label='2nd deriv')

decorate(xlabel='Frequency (Hz)',
                 ylabel='Amplitude ratio')

Оба являются фильтрами верхних частот, которые усиливают высокочастотные компоненты. Вторая производная является параболической, поэтому она больше всего усиливает самые высокие частоты. 2-я разность является хорошей аппроксимацией 2-й производной только на самых низких частотах, далее она существенно отклоняется.

# Лабораторная работа 10. Линейные стационарные системы

In [ ]:
import os


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from thinkdsp import decorate

In [ ]:
from thinkdsp import read_wave

response = read_wave('E:\wav\code_180960__kleeb__gunshot.wav')

start = 0.12
response = response.segment(start=start)
response.shift(-start)

response.truncate(2**16)
response.zero_pad(2**17)

response.normalize()
response.plot()
decorate(xlabel='Time (sec)')

In [ ]:
transfer = response.make_spectrum()
transfer.plot()
decorate(xlabel='Frequency (Hz)', ylabel='Amplitude')

In [ ]:
violin = read_wave('E:\wav\code_92002__jcveliz__violin-origional.wav')

start = 0.11
violin = violin.segment(start=start)
violin.shift(-start)

violin.truncate(2**16)
violin.zero_pad(2**17)

violin.normalize()
violin.plot()
decorate(xlabel='Time (sec)')

In [ ]:
spectrum = violin.make_spectrum()

In [ ]:
output = (spectrum * transfer).make_wave()
output.normalize()

In [ ]:
output.plot()

In [ ]:
output.make_audio()

In [ ]:
response.truncate(2**16)
response.plot()

In [ ]:
violin.truncate(2**16)
violin.plot()

In [ ]:
output2 = violin.convolve(response)

In [ ]:
output2.plot()

In [ ]:
output2.make_audio()

In [ ]:
len(output), len(output2)

In [ ]:
from thinkdsp import Wave

import scipy.signal
ys = scipy.signal.fftconvolve(violin.ys, response.ys)
output3 = Wave(ys, framerate=violin.framerate)

In [ ]:
output3.plot()

In [ ]:
output3.make_audio()

In [ ]:
output2.max_diff(output3)

Упражнение 2

In [ ]:
response = read_wave('E:\wav\code_stalbans_a_mono.wav')

start = 0
duration = 5
response = response.segment(duration=duration)
response.shift(-start)

response.normalize()
response.plot()
decorate(xlabel='Time (sec)')

In [ ]:
response.make_audio()

In [ ]:
transfer = response.make_spectrum()
transfer.plot()
decorate(xlabel='Frequency (Hz)', ylabel='Amplitude')

In [ ]:
transfer.plot()
decorate(xlabel='Frequency (Hz)', ylabel='Amplitude',
         xscale='log', yscale='log')

In [ ]:
wave = read_wave('E:\wav\code_170255__dublie__trumpet.wav')

start = 0.0
wave = wave.segment(start=start)
wave.shift(-start)

wave.truncate(len(response))
wave.normalize()
wave.plot()
decorate(xlabel='Time (sec)')

In [ ]:
wave.make_audio()

In [ ]:
spectrum = wave.make_spectrum()

In [ ]:
len(spectrum.hs), len(transfer.hs)

In [ ]:
spectrum.fs

In [ ]:
transfer.fs

In [ ]:
wave.plot()

In [ ]:
output.plot()

In [ ]:
output.make_audio()

In [ ]:
convolved2 = wave.convolve(response)
convolved2.normalize()
convolved2.make_audio()

# Лабораторная работа 11. Модуляция и выборка

In [ ]:
import os

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from thinkdsp import decorate

In [ ]:
from thinkdsp import read_wave

wave = read_wave('E:\wav\code_263868__kevcio__amen-break-a-160-bpm.wav')
wave.normalize()
wave.plot()

In [ ]:
wave.make_audio()

In [ ]:
spectrum = wave.make_spectrum(full=True)
spectrum.plot()

In [ ]:
factor = 3
framerate = wave.framerate / factor
cutoff = framerate / 2 - 1

In [ ]:
spectrum.low_pass(cutoff)
spectrum.plot()

Звук после фильтрации

In [ ]:
filtered = spectrum.make_wave()
filtered.make_audio()

In [ ]:
from thinkdsp import Wave

def sample(wave, factor):

    ys = np.zeros(len(wave))
    ys[::factor] = np.real(wave.ys[::factor])
    return Wave(ys, framerate=wave.framerate)

In [ ]:
sampled = sample(filtered, factor)
sampled.make_audio()

In [ ]:
sampled_spectrum = sampled.make_spectrum(full=True)
sampled_spectrum.plot()

In [ ]:
sampled_spectrum.low_pass(cutoff)
sampled_spectrum.plot()

In [ ]:
sampled_spectrum.scale(factor)
spectrum.plot()
sampled_spectrum.plot()

In [ ]:
spectrum.max_diff(sampled_spectrum)

In [ ]:
interpolated = sampled_spectrum.make_wave()
interpolated.make_audio()

In [ ]:
filtered.plot()
interpolated.plot()

In [ ]:
filtered.max_diff(interpolated)